# Download NDVI indices yearwise in Geotiff format

## Import packages

In [42]:
import geemap
import ee 
import os
import geemap.colormaps as cm
# from matplotlib import cm
import warnings
warnings.filterwarnings('ignore')

In [43]:
#get the variable initialized
%store -r 

In [44]:
# ee.Authenticate()

Opening in existing browser session.
Enter verification code: 4/1ARtbsJpmjFxhyqJcDUH0wY8R8wsH39jEzZW_qfWd5-SW7r2e9235lZwtG3Y

Successfully saved authorization token.


In [45]:
# A Map object is created which helps to interact with a open street interactive map and add layers to it.
Map = geemap.Map(center=[40,100], zoom=4)

In [46]:
Map

Map(center=[40, 100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

## Get the dataset from GEE server

In [47]:
# The landsat 7 data is called from the Google Earth Engine Server and a Image Collection is created.
dataset_l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_RT_TOA')

# Run the code below to get more information about the dataset we want.
# print(dataset_l7.size().getInfo()) 

## Helper Functions

### Spectral Index Computations

In [48]:
def ndvi_image_compute(img):
    '''
    Function to compute the ndvi index of the averaged image.It also initializes a download ready ndvi pallete for
    visualization and returns the final color image (with pallete)/.
    '''
    # Computes the nvdi index of the image which is (Band 4 – Band 3) / (Band 4 + Band 3) for LandSAT 7.
    ndvi_img = img.normalizedDifference(['B4', 'B3'])
    ndvi_visualization = {
      'palette': cm.palettes.ndvi,
        }
    # Get a corrected image with the pallete specified.
    corrected_img = ndvi_img.visualize(**ndvi_visualization)
    
    return corrected_img

In [49]:
def ndwi_image_compute(img):
    '''
    Function to compute the ndwi index of the averaged image.It also initializes a download ready ndwi pallete for
    visualization and returns the final color image (with pallete)/.
    '''
    # Computes the nvdi index of the image which is (Band 4 – Band 5) / (Band 4 + Band 5) for LandSAT 7.
    ndwi_img = img.normalizedDifference(['B4', 'B5'])
    ndwi_visualization = {
      'palette': cm.palettes.ndwi,
        }
    # Get a corrected image with the pallete specified.
    corrected_img = ndwi_img.visualize(**ndwi_visualization)
    
    return corrected_img

In [50]:
def ndbi_image_compute(img):
    '''
    Function to compute the ndwi index of the averaged image.It also initializes a download ready ndwi pallete for
    visualization and returns the final color image (with pallete)/.
    '''
    # Computes the nvdi index of the image which is (Band 5 – Band 4) / (Band 5 + Band 4) for LandSAT 7.
    ndbi_img = img.normalizedDifference(['B5', 'B4'])
    ndbi_visualization = {
      'min': 0.5, 
      'max': 1, 
      'palette': ['00FFFF', '0000FF'],
        }
    # Get a corrected image with the pallete specified.
    corrected_img = ndbi_img.visualize(**ndbi_visualization)
    
    return corrected_img

In [51]:
def mndwi_image_compute(img):
    '''
    Function to compute the ndwi index of the averaged image.It also initializes a download ready ndwi pallete for
    visualization and returns the final color image (with pallete)/.
    '''
    # Computes the nvdi index of the image which is (Band 2 – Band 5) / (Band 2 + Band 5) for LandSAT 7.
    mndwi_img = img.normalizedDifference(['B2', 'B5'])
    mndwi_visualization = {
      'palette': cm.palettes.mndwi,
        }
    # Get a corrected image with the pallete specified.
    corrected_img = ndbi_img.visualize(**mndwi_visualization)
    
    return corrected_img

### Other processing functions

In [52]:
def clip_to_shape(image):
    '''
    Function to take an image and clip it according to the region of interest(roi).
    It takes an image as an input and outputs the clipped version of the image according to roi
    '''
    return image.clip(roi)

In [53]:
def maskclouds(image):
    '''
    Function to take an image and mask the clouds for better visibility. The returns the masked image.
    '''
    # Select the BQA band, which has the cloud information
    qa = image.select('BQA')

    # In BQA Band bit 4 is the cloud band, hence activate only that bit, set all other bits of BQA to zero
    cloudBit = 1 << 4 #(00000001 -> 00010000) 

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBit).eq(0) # And with all the other bits of BQA to check for clouds
    return image.updateMask(mask)

In [54]:
def filter_date(year,i,flag):
    '''
    Function to compute the start and the end date of a particular season. It takes the year and the season of 
    which we need to find the date and returns the start and the end date is YYYY/MM/DD format
    '''
    start_date = ee.Date.fromYMD(year,seasons[i][1],seasons[i][0])
    if flag==0:
        end_date = ee.Date.fromYMD(year,seasons[i][3],seasons[i][2])
    else:
        end_date = ee.Date.fromYMD(year+1,seasons[i][3],seasons[i][2])
    
    return start_date, end_date

In [55]:
def get_filtered_image(year,i,roi,spectral_index,flag):
    '''
    Function to get the final computed image of the year specified ( sort by filterdate_ cloud_masked + mean +
    ndvi computed)/. The function returns the final computed image.
    '''
    start_date,end_date = filter_date(year,i,flag)
    
    img = (dataset_l7.filterDate(start_date,end_date)
                    .filterBounds(roi)
                    .map(maskclouds))
    if spectral_index == 'NDVI':
        img = img.select('B3','B4').mean()
        img = ndvi_image_compute(img)
    elif spectral_index =='NDWI':
        img =img.select('B2','B4').mean()
        img = ndwi_image_compute(img)
    elif spectral_index =='NDBI':
        img =img.select('B5','B4').mean()
        img = ndbi_image_compute(img)
    elif spectral_index =='MNDWI':
        img = img.select('B2','B5').mean()
        img = mndwi_image_compute(img)
    return img
    

In [65]:
def yearlyMap(year,district_name,roi,spectral_index):
    '''
    Function to loop over the five selected intervals over an year by taking a year input.
    '''
    year_folder = spectral_index + '-' + district_name
    for k,season in enumerate(seasons):
        if season=='djf' or season== 'ann':
            img = get_filtered_image(year,season,roi,spectral_index,flag=1) 
        else:
            img = get_filtered_image(year,season,roi,spectral_index,flag=0)
        filename = season + '-' + str(year)

        download_to_drive(filename, year_folder,img,roi)
        # print(year_folder)
        # print(filename)

In [66]:
def download_to_drive(filename,year_folder,img,roi):
    '''
    Function to download the images year wise into the authenticated drive in a geotiff format in the specified
    year folders.
    '''
    task = ee.batch.Export.image.toDrive(image= img.clip(roi),
                                     description='spectral index computation for two decades',
                                     scale=30,
                                     region=roi.geometry(),
                                     folder=year_folder,
                                     fileNamePrefix=filename,
                                     fileFormat='GeoTIFF')
    task.start()

## Looping code

In [67]:
# List of years
years = [y for y in range(start_year,end_year)]

In [1]:
for roi,district_name in shape_files.items():
    # Loop the computation over the specified list of years for this roi
    for year in years:
        yearlyMap(year,district_name,roi,spectral_index)

In [60]:
spectral_index

'NDVI'